In [1]:
pip install pytesseract pillow opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
print(pytesseract.get_tesseract_version())  # Should return Tesseract version


5.5.0.20241111


In [3]:
import cv2
import pytesseract
from PIL import Image

# Set Tesseract path (Only for Windows)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Load image
image = cv2.imread(r"C:\Users\singh\Downloads\loan-approval-prediction-main\loan-approval-prediction-main\ocrver\AGAM adhaar sample.png")

# Convert image to grayscale (improves OCR accuracy)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Optional: Apply thresholding
gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

# Save preprocessed image
cv2.imwrite("processed.png", gray)

# Extract text using Tesseract
text = pytesseract.image_to_string(gray, lang="eng")  # Change language if needed

print("Extracted Text:\n", text)

Extracted Text:
 A TETFT
‘GOVERNMENT OF INDIA

Name AGAM
DOB: 22/01/2005

Gender: FEMALE

0000 1111 2222 BF:
EE# & AAFR




In [4]:
# import libraries
import csv
import cv2
import pytesseract


def pre_processing(image):
    """
    This function take one argument as
    input. this function will convert
    input image to binary image
    :param image: image
    :return: thresholded image
    """
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # converting it to binary image
    threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    # saving image to view threshold image
    cv2.imwrite('thresholded.png', threshold_img)

    cv2.imshow('threshold image', threshold_img)
    # Maintain output window until
    # user presses a key
    cv2.waitKey(0)
    # Destroying present windows on screen
    cv2.destroyAllWindows()

    return threshold_img


def parse_text(threshold_img):
    """
    This function take one argument as
    input. this function will feed input
    image to tesseract to predict text.
    :param threshold_img: image
    return: meta-data dictionary
    """
    # configuring parameters for tesseract
    tesseract_config = r'--oem 3 --psm 6'
    # now feeding image to tesseract
    details = pytesseract.image_to_data(threshold_img, output_type=pytesseract.Output.DICT,
                                        config=tesseract_config, lang='eng')
    return details


def draw_boxes(image, details, threshold_point):
    """
    This function takes three argument as
    input. it draw boxes on text area detected
    by Tesseract. it also writes resulted image to
    your local disk so that you can view it.
    :param image: image
    :param details: dictionary
    :param threshold_point: integer
    :return: None
    """
    total_boxes = len(details['text'])
    for sequence_number in range(total_boxes):
        if int(details['conf'][sequence_number]) > threshold_point:
            (x, y, w, h) = (details['left'][sequence_number], details['top'][sequence_number],
                            details['width'][sequence_number], details['height'][sequence_number])
            image = cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # saving image to local
    cv2.imwrite('captured_text_area.png', image)
    # display image
    cv2.imshow('captured text', image)
    # Maintain output window until user presses a key
    cv2.waitKey(0)
    # Destroying present windows on screen
    cv2.destroyAllWindows()


def format_text(details):
    """
    This function take one argument as
    input.This function will arrange
    resulted text into proper format.
    :param details: dictionary
    :return: list
    """
    parse_text = []
    word_list = []
    last_word = ''
    for word in details['text']:
        if word != '':
            word_list.append(word)
            last_word = word
        if (last_word != '' and word == '') or (word == details['text'][-1]):
            parse_text.append(word_list)
            word_list = []

    return parse_text


def write_text(formatted_text):
    """
    This function take one argument.
    it will write arranged text into
    a file.
    :param formatted_text: list
    :return: None
    """
    with open('resulted_text.txt', 'w', newline="") as file:
        csv.writer(file, delimiter=" ").writerows(formatted_text)


if __name__ == "__main__":
    # reading image from local
    image = cv2.imread(r'C:\Users\singh\Downloads\loan-approval-prediction-main\loan-approval-prediction-main\ocrver\AGAM adhaar sample.png')
    # calling pre_processing function to perform pre-processing on input image.
    thresholds_image = pre_processing(image)
    # calling parse_text function to get text from image by Tesseract.
    parsed_data = parse_text(thresholds_image)
    # defining threshold for draw box
    accuracy_threshold = 30
    # calling draw_boxes function which will draw dox around text area.
    draw_boxes(thresholds_image, parsed_data, accuracy_threshold)
    # calling format_text function which will format text according to input image
    arranged_text = format_text(parsed_data)
    # calling write_text function which will write arranged text into file
    write_text(arranged_text)

In [6]:
import cv2
import pytesseract

def extract_text_from_aadhaar(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    text = pytesseract.image_to_string(gray)
    return text

text = extract_text_from_aadhaar(r"C:\Users\singh\Downloads\loan-approval-prediction-main\loan-approval-prediction-main\ocrver\AGAM adhaar sample.png")
print(text)

S TR
\GOVERNMENT OF INDIA|

Name AGAM
DOB: 22/01/2005

Gender: FEMALE

0000 1111 2222 EK
EE# & HOHR




In [ ]:

# import re

# def extract_aadhaar_details(text):
#     name_pattern = r"(?i)(?:name|naam)[:\s]*([A-Za-z ]+)"
#     dob_pattern =   r"\b\d{2}/\d{2}/\d{4}\b"
#     aadhaar_pattern = r"\b\d{4} \d{4} \d{4}\b"
#     name = re.search(name_pattern, text)
#     dob = re.search(dob_pattern, text)
#     aadhaar_number = re.search(aadhaar_pattern, text)

#     return {
#         "name": name.group(1) if name else None,
#         "dob": dob.group(1) if dob else None,
#         "aadhaar_number": aadhaar_number.group() if aadhaar_number else None
#     }

# extracted_data = extract_aadhaar_details(text)
# print(extracted_data)

In [7]:
import re

def extract_aadhaar_details(text):
    # Pattern to extract name (assumes "Name: XYZ" or similar format)
    name_pattern = r"(?i)(?:name|naam)[:\s]*([A-Za-z ]+)"
    
    # Pattern to extract gender (Male, Female, Transgender)
    gender_pattern = r"\b(?:Male|Female)\b"

    name = re.search(name_pattern, text)
    gender = re.search(gender_pattern, text, re.IGNORECASE)

    return {
        "name": name.group(1).strip() if name else None,
        "gender_number": gender.group() if gender else None
    }

# Extract details
extracted_data = extract_aadhaar_details(text)
print(extracted_data)


{'name': 'AGAM', 'gender_number': 'FEMALE'}
